In [2]:
import getpass

Download neccessnary libary

In [ ]:
# ! pip install langchain langchain_community langchainhub


In [3]:
#kiem tra xem co cuda khong (gpu)
import torch
device = 0 if torch.cuda.is_available() else -1
device

-1

In [1]:
#danh sach cac model 

#TinyLlama/TinyLlama_v1.1
#TinyLlama/TinyLlama-1.1B-Chat-v1.0
#meta-llama/Llama-3.2-1B-Instruct
#meta-llama/Llama-3.2-1B
huggingface_model_name = "meta-llama/Llama-3.2-1B-Instruct"
embedding_model_name = "all-MPNet-base-v2"

In [ ]:
# %pip install transformers

In [86]:
import subprocess
huggingface_api_key = getpass.getpass()
huggingface_login_result = subprocess.run(["huggingface-cli", "login", "--token", huggingface_api_key])

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
The token `learning` has been saved to /home/bach/.cache/huggingface/stored_tokens
Your token has been saved to /home/bach/.cache/huggingface/token
Login successful.
The current active token is: `learning`


In [2]:
from transformers import AutoTokenizer, pipeline
tokenizer = AutoTokenizer.from_pretrained(huggingface_model_name)

pipe = pipeline("text-generation",model = huggingface_model_name)

from langchain_huggingface import HuggingFaceEndpoint,ChatHuggingFace

llm = HuggingFaceEndpoint(
    repo_id=huggingface_model_name,
    
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03,
)

chat = ChatHuggingFace(llm=llm, verbose=True)

# model = AutoModelForCausalLM.from_pretrained(model_name)

# prompt = "can you speak vietnamese ?"
# inputs = tokenizer(prompt, return_tensors="pt")
# generate_ids = model.generate(**inputs, max_length=30)
# tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

Device set to use cpu


INDEXING

Load documents

In [ ]:
# # install poppler tesseract libmagic
# !pip install  "unstructured[all-docs]" langchain-unstructured unstructured-client unstructured


# #cai them pdf2image pytesseract python-magic
# #unstructured-client can tai neu muon dung api de goi toi unstructured
# # unstructured can tai neu muon dung local

In [6]:
from langchain_unstructured import UnstructuredLoader
#### INDEXING ####

# Load Documents
loader = UnstructuredLoader(
    file_path=["../../../bach/WINDOWS 10/Users/HOME PC/Downloads/Principles_methods_and_Evaluation.pdf",
               "../../../bach/WINDOWS 10/Users/HOME PC/Downloads/LLM.pdf"],
    strategy="hi_res",
)
docs = loader.load()

INFO: pikepdf C++ to Python logger bridge initialized
INFO: Reading PDF for file: ../../../bach/WINDOWS 10/Users/HOME PC/Downloads/Principles_methods_and_Evaluation.pdf ...
INFO: Reading PDF for file: ../../../bach/WINDOWS 10/Users/HOME PC/Downloads/LLM.pdf ...


split document into chunk

In [35]:

print(docs[0].metadata)
print(docs[5085].metadata)

{'source': '../../../bach/WINDOWS 10/Users/HOME PC/Downloads/Principles_methods_and_Evaluation.pdf', 'detection_class_prob': 0.8201810717582703, 'coordinates': {'points': ((np.float64(125.29678344726562), np.float64(92.36564502777784)), (np.float64(125.29678344726562), np.float64(114.59364725000019)), (np.float64(595.765625), np.float64(114.59364725000019)), (np.float64(595.765625), np.float64(92.36564502777784))), 'system': 'PixelSpace', 'layout_width': 1654, 'layout_height': 2205}, 'links': [{'text': 'Egyptian Informatics Journal ( 2015 ) 16 , 261', 'url': 'http://dx.doi.org/10.1016/j.eij.2015.06.005', 'start_index': 0}, {'text': '273', 'url': 'http://dx.doi.org/10.1016/j.eij.2015.06.005', 'start_index': 44}], 'last_modified': '2025-03-15T11:06:23', 'filetype': 'application/pdf', 'languages': ['eng'], 'page_number': 1, 'file_directory': '../../../bach/WINDOWS 10/Users/HOME PC/Downloads', 'filename': 'Principles_methods_and_Evaluation.pdf', 'category': 'Header', 'element_id': 'afdb052

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores.utils import filter_complex_metadata
# Split
text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(tokenizer=tokenizer,chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
filtered_metadatas = filter_complex_metadata(splits)

In [ ]:
print(filtered_metadatas[100].metadata)
print(splits[100].metadata)

{'source': '../../../bach/WINDOWS 10/Users/HOME PC/Downloads/Principles_methods_and_Evaluation.pdf', 'detection_class_prob': 0.9542631506919861, 'last_modified': '2025-03-15T11:06:23', 'filetype': 'application/pdf', 'page_number': 6, 'parent_id': 'fdc1c0ae4ae77f00ffd79e600dbaad83', 'file_directory': '../../../bach/WINDOWS 10/Users/HOME PC/Downloads', 'filename': 'Principles_methods_and_Evaluation.pdf', 'category': 'NarrativeText', 'element_id': '2cd6b7f101382a9fcb9a2aec219fb2c7'}
{'source': '../../../bach/WINDOWS 10/Users/HOME PC/Downloads/Principles_methods_and_Evaluation.pdf', 'detection_class_prob': 0.9542631506919861, 'last_modified': '2025-03-15T11:06:23', 'filetype': 'application/pdf', 'page_number': 6, 'parent_id': 'fdc1c0ae4ae77f00ffd79e600dbaad83', 'file_directory': '../../../bach/WINDOWS 10/Users/HOME PC/Downloads', 'filename': 'Principles_methods_and_Evaluation.pdf', 'category': 'NarrativeText', 'element_id': '2cd6b7f101382a9fcb9a2aec219fb2c7'}


In [ ]:
# !pip install "pymongo[srv]" langchain-mongodb

In [ ]:
#connect to mongoDB server
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
user_name = getpass.getpass()
password = getpass.getpass()
app_name = "******"
cluster_name = "***"
uri = f"mongodb+srv://{user_name}:{password}@{cluster_name}.lzb32h6.mongodb.net/?retryWrites=true&w=majority&appName={app_name}"
# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))
# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


In [ ]:
#init vector_store
from langchain_mongodb import MongoDBAtlasVectorSearch
from langchain.embeddings import HuggingFaceEmbeddings

database_name = "learning"
collection_name = "learning_rag"

embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)
db = client[database_name]
MONGODB_COLLECTION = db[collection_name]
ATLAS_VECTOR_SEARCH_INDEX_NAME = "vector_search_index"

vector_store = MongoDBAtlasVectorSearch(
    embedding=embeddings,
    collection=MONGODB_COLLECTION,
    index_name=ATLAS_VECTOR_SEARCH_INDEX_NAME,
    relevance_score_fn="cosine",
)
# vector_store = MongoDBAtlasVectorSearch.from_connection_string(
#     connection_string=uri,
#     namespace="db_name.collection_name",
#     embedding=embeddings,
#     index_name="vector_index",
# )

/tmp/ipykernel_19304/2865424260.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)
INFO: Use pytorch device_name: cpu
INFO: Load pretrained SentenceTransformer: all-MPNet-base-v2


In [36]:
#add document to vector store (luu document vao database)
vector_store.add_documents(filtered_metadatas)

['68160e444384817b2c24c0e8',
 '68160e444384817b2c24c0e9',
 '68160e444384817b2c24c0ea',
 '68160e444384817b2c24c0eb',
 '68160e444384817b2c24c0ec',
 '68160e444384817b2c24c0ed',
 '68160e444384817b2c24c0ee',
 '68160e444384817b2c24c0ef',
 '68160e444384817b2c24c0f0',
 '68160e444384817b2c24c0f1',
 '68160e444384817b2c24c0f2',
 '68160e444384817b2c24c0f3',
 '68160e444384817b2c24c0f4',
 '68160e444384817b2c24c0f5',
 '68160e444384817b2c24c0f6',
 '68160e444384817b2c24c0f7',
 '68160e444384817b2c24c0f8',
 '68160e444384817b2c24c0f9',
 '68160e444384817b2c24c0fa',
 '68160e444384817b2c24c0fb',
 '68160e444384817b2c24c0fc',
 '68160e444384817b2c24c0fd',
 '68160e444384817b2c24c0fe',
 '68160e444384817b2c24c0ff',
 '68160e444384817b2c24c100',
 '68160e444384817b2c24c101',
 '68160e444384817b2c24c102',
 '68160e444384817b2c24c103',
 '68160e444384817b2c24c104',
 '68160e444384817b2c24c105',
 '68160e444384817b2c24c106',
 '68160e444384817b2c24c107',
 '68160e444384817b2c24c108',
 '68160e444384817b2c24c109',
 '68160e444384

In [45]:
#create vector search index
from pymongo.operations import SearchIndexModel
import time
# Access your database and collection
database = client[database_name]
collection = database[collection_name]
# Create your index model, then create the search index
search_index_model = SearchIndexModel(
  definition={
    "fields": [
      {
        "type": "vector",
        "path": "embedding",
        "numDimensions": 768,
        "similarity": "cosine",
      }
    ]
  },
  name=ATLAS_VECTOR_SEARCH_INDEX_NAME,
  type="vectorSearch"
)
result = collection.create_search_index(model=search_index_model)
print("New search index named " + result + " is building.")
# Wait for initial sync to complete
print("Polling to check if the index is ready. This may take up to a minute.")
predicate=None
if predicate is None:
  predicate = lambda index: index.get("queryable") is True
while True:
  indices = list(collection.list_search_indexes(result))
  if len(indices) and predicate(indices[0]):
    break
  time.sleep(5)
print(result + " is ready for querying.")


New search index named vector_search_index is building.
Polling to check if the index is ready. This may take up to a minute.
vector_search_index is ready for querying.


In [ ]:
from langchain import hub
import os
langchain_api_key = getpass.getpass()

os.environ["LANGSMITH_TRACING"]="true"
os.environ["LANGSMITH_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGSMITH_PROJECT"]="learning_rag"
os.environ["LANGSMITH_API_KEY"] = langchain_api_key

prompt = hub.pull("rlm/rag-prompt",api_key=langchain_api_key)

example_messages = prompt.invoke(
    {"context": "(context goes here)", "question": "(question goes here)"}
).to_messages()

assert len(example_messages) == 1
print(example_messages[0].content)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: (question goes here) 
Context: (context goes here) 
Answer:


Failed to send compressed multipart ingest: Connection error caused failure to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. Please confirm your internet connection. ConnectionError(MaxRetryError('HTTPSConnectionPool(host=\'api.smith.langchain.com\', port=443): Max retries exceeded with url: /runs/multipart (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7150ccc76800>: Failed to resolve \'api.smith.langchain.com\' ([Errno -3] Temporary failure in name resolution)"))'))
Content-Length: 984
API Key: lsv2_********************************************edtrace=efef38eb-f704-49b9-b901-01c62f00b926,id=efef38eb-f704-49b9-b901-01c62f00b926; trace=efef38eb-f704-49b9-b901-01c62f00b926,id=efef38eb-f704-49b9-b901-01c62f00b926


In [ ]:
print(prompt.format(context = "context goes here",question = "question goes here"))

Human: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: question goes here 
Context: context goes here 
Answer:


Create state and node to build graph in lang graph

In [ ]:
# %pip install langchain_huggingface

In [ ]:
from langchain_core.documents import Document
from typing_extensions import List, TypedDict
from langchain_huggingface import HuggingFacePipeline

from huggingface_hub import InferenceClient

client_huggingface = InferenceClient(
    provider="sambanova",
    api_key=huggingface_api_key,
)

completion = client_huggingface.chat.completions.create(
    model="meta-llama/Llama-3.2-1B-Instruct",
    messages=[
        {
            "role": "user",
            "content": "Hi"
        }
    ],
    max_tokens=512,
)

# llm = HuggingFacePipeline(pipeline=pipe)

class State(TypedDict):
    question: str
    context: List[Document]
    answer: str

def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = client.chat.completions.create(
        model=huggingface_model_name,
        messages=[{
            "role": "user",
            "content":messages.to_messages()[0].content
        }]
    )
    return {"answer" : response.choices[0].message.content}
    # response = llm.invoke(messages)
    # return {"answer": response}

compile graph

In [ ]:
#%pip install langgraph

In [104]:
from langgraph.graph import START, StateGraph

graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [70]:
question = "What is GroupLens"

In [105]:
result = graph.invoke({"question": question})

print(f'Context: {result["context"]}\n\n')
print(f'Answer: {result["answer"]}')

Context: [Document(id='68160e444384817b2c24c201', metadata={'_id': '68160e444384817b2c24c201', 'source': '../../../bach/WINDOWS 10/Users/HOME PC/Downloads/Principles_methods_and_Evaluation.pdf', 'detection_class_prob': 0.9298052787780762, 'last_modified': '2025-03-15T11:06:23', 'filetype': 'application/pdf', 'page_number': 13, 'parent_id': '303c78b4b9249a87b3e966e233f25538', 'file_directory': '../../../bach/WINDOWS 10/Users/HOME PC/Downloads', 'filename': 'Principles_methods_and_Evaluation.pdf', 'category': 'ListItem', 'element_id': '325238850e5be70b2fdf9d723b04d86a'}, page_content='[86] Sarwar BM, Konstan JA, Herlocker JL, Miller B, Riedl JT. Using ﬁltering agents to improve prediction quality in the grouplens research, collaborative ﬁltering system. In: Proceedings of the ACM conference on computer supported cooperative work. New York (NY, USA): ACM; 1998. p. 345–54.'), Document(id='68160e444384817b2c24c1bf', metadata={'_id': '68160e444384817b2c24c1bf', 'source': '../../../bach/WINDO

doan duoi nay la testing nghich nghich thoi khong co gi dau


In [106]:
for message, metadata in graph.stream(
    {"question": question}, stream_mode="messages"
):
    print(message.content, end="|")

In [ ]:
from langchain_huggingface import HuggingFaceEndpoint,ChatHuggingFace

llm = HuggingFaceEndpoint(
    repo_id=huggingface_model_name,
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03,
)

In [94]:
from huggingface_hub import InferenceClient
# import getpass 
# huggingface_api_key = getpass.getpass()
client_huggingface = InferenceClient(
    provider="sambanova",
    api_key=huggingface_api_key,
)

completion = client_huggingface.chat.completions.create(
    model="meta-llama/Llama-3.2-1B-Instruct",
    messages=[
        {
            "role": "user",
            "content": "hi"
        }
    ],
    max_tokens=512,
)

print(completion.choices[0].message.content)

How can I assist you today?


Dung (ai/human/system)MessagePromptTemplate de tao 1 template prompt don le  

Neu muon ket hop cac loai template nay lai thi phai dung ChatpromptTemplate

In [71]:
from langchain_core.prompts import AIMessagePromptTemplate,HumanMessagePromptTemplate,SystemMessagePromptTemplate,PromptTemplate,ChatPromptTemplate

ai_prompt = AIMessagePromptTemplate(prompt=PromptTemplate(
    input_variables="ai_message",
    template="ai message is : {ai_message}"
))
prompt_test1 = ChatPromptTemplate.from_messages([ai_prompt])
prompt_test1.invoke({"ai_message": "hello"}).to_messages()[0].content

'ai message is : hello'

In [95]:
ai_prompt2 = AIMessagePromptTemplate.from_template("ai_message : {ai_message}")
human_prompt3 = HumanMessagePromptTemplate.from_template("human : {human}")
prompt_test2 = ChatPromptTemplate.from_messages([ai_prompt2,human_prompt3])
prompt_test2.invoke({"ai_message": "hello","human":"human"}).to_messages()

[AIMessage(content='ai_message : hello', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='human : human', additional_kwargs={}, response_metadata={})]

In [100]:
template = [
    ("user", "{human}"),
    ("ai", "{ai_message}")
]

prompt_test2 = ChatPromptTemplate.from_messages(template)
demo_test = prompt_test2.format_messages(ai_message = "hi",human = "oi")
dict_messages = [
    {"role": m.type, "content": m.content} for m in demo_test
]

In [101]:
dict_messages

[{'role': 'human', 'content': 'oi'}, {'role': 'ai', 'content': 'hi'}]

test streaming ouput

In [3]:
from transformers import TextStreamer
from langchain_huggingface import HuggingFacePipeline
streamer = TextStreamer(tokenizer)
pipe_demo = pipeline(model=huggingface_model_name,
                tokenizer=tokenizer, 
                streamer=streamer)
llm = HuggingFacePipeline(pipeline=pipe)

Device set to use cpu


In [ ]:
import os
import getpass
os.environ["TAVILY_API_KEY"] = getpass.getpass()

In [8]:
from langchain_community.tools import TavilySearchResults

tool = TavilySearchResults(
    max_results=5,
    include_answer=True,
    include_raw_content=True,
    include_images=True,
    # search_depth="advanced",
    # include_domains = []
    # exclude_domains = []
)

In [ ]:
from transformers import pipeline
from langchain_huggingface import HuggingFacePipeline

pipe = pipeline(model="meta-llama/Llama-3.2-1B-Instruct")
llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
llm.invoke("who are you ?")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'who are you ? \nYou are a person who is in a relationship with someone who is not your spouse. \nThis'

In [11]:
llm.invoke("who are you ?",
{"do_sample" : True, 
    "top_p":0.9,
    "top_k":50,
    "temperature":0.1,
    "max_new_tokens":100})

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


"who are you ? I'm a student at a university, and I'm a bit of an introvert. I like"

In [12]:
a = llm.invoke("who are you ?",
{"do_sample" : True, 
    "top_p":0.8,
    "top_k":50,
    "temperature":0.3,
    "max_new_tokens":200})

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [69]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [103]:
prompt.format_messages(context = "hi",question= "question")[0]

HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: question \nContext: hi \nAnswer:", additional_kwargs={}, response_metadata={})